In [78]:
import autoencoder
import pandas as pd
import torch

from sklearn.model_selection import KFold

## Prepare data:
#### Read joined data (pre + post treatment)

In [4]:
lincs_joined_df = pd.read_parquet("data/lincs_pretreatment.parquet")
kaggle_joined_df = pd.read_parquet("data/kaggle_pretreatment.parquet")
test_joined_df = pd.read_parquet("data/test_pretreatment.parquet")
print(f"lincs_joined_df = {lincs_joined_df.shape}\nkaggle_joined_df = {kaggle_joined_df.shape}\ntest_joined_df = {test_joined_df.shape}")

lincs_joined_df = (107404, 1842)
kaggle_joined_df = (602, 1841)
test_joined_df = (255, 921)


#### Kaggle provided data

In [8]:
de_train = pd.read_parquet('data/de_train.parquet')
id_map = pd.read_csv('data/id_map.csv',index_col='id')
print(f"de_train = {de_train.shape}\nid_map = {id_map.shape}")

de_train = (614, 18216)
id_map = (255, 2)


#### Define features of interest and sort data accordingly.

In [9]:
features = ['cell_type', 'sm_name']
multiindex_features = [("label",'cell_type'),("label",'sm_name')]

transcriptome_cols = de_train.columns[5:]
landmark_cols = kaggle_joined_df["post_treatment"].columns
print(f"transcriptome_cols = {transcriptome_cols.shape}\nlandmark_cols = {landmark_cols.shape}")

transcriptome_cols = (18211,)
landmark_cols = (918,)


In [10]:
# We only need to sort these two dataframes because they represent the same underlying dataset.
de_train = de_train.query("~control").sort_values(features)
kaggle_joined_df = kaggle_joined_df.sort_values(multiindex_features)
# Sanity check that these dfs align.
genes_align = (kaggle_joined_df["post_treatment"] == de_train[landmark_cols]).all(axis=None)
labels_align = (kaggle_joined_df["label"][features] == de_train[features]).all(axis=None)
genes_align and labels_align

True

#### CV splits

In [80]:
eval_cells_only_df = kaggle_joined_df[kaggle_joined_df["label"]["cell_type"].isin(["B cells", "Myeloid cells"])][multiindex_features]
len(eval_cells_only_df)

30

In [84]:
fold_to_eval_df = {}
skf = KFold(n_splits=3, random_state=42, shuffle=True)
for i,(t,v) in enumerate(skf.split(eval_cells_only_df)):
    fold_to_eval_df[i] = eval_cells_only_df.iloc[v]

for i, df in fold_to_eval_df.items():
    print(f"fold = {i} of shape {df.shape}")

fold = 0 of shape (10, 2)
fold = 1 of shape (10, 2)
fold = 2 of shape (10, 2)


In [87]:
def make_mask(fold):
    val = fold_to_eval_df[fold]
    return kaggle_joined_df[("label","sm_name")].isin(val[("label","sm_name")]) & kaggle_joined_df[("label","cell_type")].isin(val[("label","cell_type")])

print("Using fold 0 as validation set:")
print(f"Train data = {pd.concat([kaggle_joined_df[~make_mask(0)],lincs_joined_df]).shape}")
print(f"Validation data = {kaggle_joined_df[make_mask(0)].shape}")

Using fold 0 as validation set:
Train data = (107994, 1843)
Validation data = (12, 1841)


In [88]:
class Translator(torch.nn.Module):
    def __init__(self,config):
        super(Translator,self).__init__()
        # This will eventually be changed to a GNN
        self.pert_embed = torch.nn.Embedding(
            len(column_uniques["pert_iname"]), config["pert_emb_size"])

        # This needs to be able to handle out of dictionary
        self.cell_embed = torch.nn.Embedding(
            len(column_uniques["cell_id"]), config["cell_emb_size"])

        self.config = config
        input_dim = config["pert_emb_size"] + config["cell_emb_size"] + config["latent_dim"]
        self.translation = utils.make_sequential(input_dim,config["hidden_dim"],config["latent_dim"],config["dropout"])

    def forward(self,inp,z):
        pemb = self.pert_embed(inp["pert_iname"])
        cemb = self.cell_embed(inp["cell_id"])
        x = torch.cat((pemb, cemb, z), dim=1)
        return self.translation(x)